In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

from functions import *

url = "http://www.cinemaonline.sg/movies/nowshowing.aspx"
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")
tags = soup.findAll("li")
tags = [str(i) for i in tags]

movieData = []
for i in range(0, len(tags)):
    movieData.append(striphtml(tags[i]))

del movieData[:20]
movieData = movieData[:len(movieData)-4]

for i in range(0, len(movieData)):
    movieData[i] = stripStuff(movieData[i])

duration = []
movieName = []
classification = []
castList = []
releaseDate = []

for i in movieData:
    find = re.search("Running Time: (.+?)Release", i).group(1)
    duration.append(find)
    find = re.search("(.+?)\(", i).group(1)
    movieName.append(find)
    find = re.search("Classification: (.+?)Genre:", i).group(1)
    classification.append(find)
    find = re.search("Cast: (.+?)\r", i).group(1)
    castList.append(find)
    find = re.search("Release Date: (.+?)Language", i).group(1)
    releaseDate.append(find)
    
for i in range(0, len(movieName)):
    movieName[i] = stripWhitespace(movieName[i])

ID = []

for i in range(1, len(movieData)+1):
    if i<10:
        ID.append("1000" + str(i))
    else:
        ID.append("100" + str(i))

viewType = []
viewStatus = []

for i in range(0, len(movieName)):
    viewType.append("Digital")
    viewStatus.append("Now Showing")

movieDF = pd.DataFrame(np.column_stack([ID, movieName, castList, duration, releaseDate, classification, viewType, viewStatus]), 
                               columns = ['ID', 'MovieName', 'Casts', 'Runtime', 'ReleaseDate', 'Classification', 'ViewType', 'ViewStatus'])

In [2]:
movieDF

,ID,MovieName,Casts,Runtime,ReleaseDate,Classification,ViewType,ViewStatus
0,10001,Hitman's Wife's Bodyguard,"Ryan Reynolds, Samuel L. Jackson, Salma Hayek,...",1 Hour 40 Minutes,24 Jun 2021,M18,Digital,Now Showing
1,10002,The Conjuring: The Devil Made Me Do It,"Vera Farmiga, Patrick Wilson, Ruairi O'Connor,...",1 Hour 52 Minutes,11 Jun 2021,NC16,Digital,Now Showing
2,10003,A Quiet Place Part II,"Emily Blunt, Cillian Murphy, Millicent Simmond...",1 Hour 37 Minutes,10 Jun 2021,PG13,Digital,Now Showing
3,10004,Cruella,"Emma Stone, Emma Thompson, Paul Walter Hauser,...",2 Hours 14 Minutes,27 May 2021,PG,Digital,Now Showing
4,10005,My Love,"Greg Hsu, Zhang Ruo Nan, Kevin Yan",1 Hour 55 Minutes,06 May 2021,PG13,Digital,Now Showing
5,10006,Konpaku,"Junaidi M. Sali, Lizzie.V., Kamariah Osman",1 Hour 49 Minutes,01 Nov 2020,NC16,Digital,Now Showing
